# Iris Dataset Example - Random Forest

In [1]:
import sys

!{sys.executable} -m pip install skl2onnx==1.17.0
!{sys.executable} -m pip install onnx==1.16.1
!{sys.executable} -m pip install onnxruntime==1.18.1


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import skl2onnx
import onnx
import sklearn
import numpy
import onnxruntime as rt
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)
print(clr)

RandomForestClassifier()


In [4]:
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type,
                      target_opset=12)

with open("iris_rf.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [5]:
sess = rt.InferenceSession("iris_rf.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run(
    [label_name], {input_name: X_test.astype(numpy.float32)})[0]
print(pred_onx)

[1 0 1 0 1 2 1 0 2 2 0 1 0 0 2 0 0 0 1 1 0 1 0 1 2 0 0 1 0 1 2 1 2 0 2 1 0
 1]


In [6]:
sess.get_outputs()[0].name

'output_label'